# 安全率のラベリングとカテゴリ変数の把握

In [ ]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from sklearn import preprocessing
import japanize_matplotlib
# LightGBM
import lightgbm as lgb
import optuna
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

In [85]:
#データの読み込み
data_folder = input("データファイルのあるフォルダまでのパス")
data_folder = data_folder.rstrip()
data_folder = data_folder.replace("\\", "/") + "/"
    
file_error = data_folder + "input_errorfiles.csv"
    
file_ok_1 = data_folder + "train_data1.csv"
file_ok_2 = data_folder + "train_labels.csv"

df_ok_1 = pd.read_csv(file_ok_1,encoding="shift-jis")
df_ok_2 = pd.read_csv(file_ok_2,encoding="shift-jis")

df_error = pd.read_csv(file_error,encoding="shift-jis")
df_ok = pd.concat([df_ok_1,df_ok_2],axis = 1)

In [86]:
#エラーデータ
## 地震時を取り除く
df_error = df_error.drop(["No","堤外側水位_地震時","堤内側水位_地震時"," 震度"],axis=1)
# 安全率変数名の一覧
safety_label =  ["せん断変形破壊","滑動、支持力","根入れ部の安定","遮水効果","矢板の耐力照査","タイロッドの耐力照査","腹起しの耐力照査"]
# エラーデータには安全率のデータがないため，説明変数に欠損値として加える．
df_error[[i for i in safety_label]] = np.nan

In [90]:
df_ok

,堤体延長,堤体幅,左右壁体高さ,中詰土天端高さ,堤外区間地表面,堤内区間地表面,引張材間隔,在来地盤重量,在来地盤区分,在来地盤N値,在来地盤摩擦角,在来地盤粘着力,在来地盤変形係数,中詰め重量,中詰め区分,中詰めN値,中詰め摩擦角,中詰め粘着力,矢板型,矢板材料,引張材直径,引張材材料番号,腹起し材質,堤外側水位_常時,堤内側水位_常時,せん断変形破壊,滑動、支持力,根入れ部の安定,遮水効果,矢板の耐力照査,タイロッドの耐力照査,腹起しの耐力照査,矢板の全長,引張材位置,腹起し番号,根入れ深さ
0,20,5.7,4.0,3.9,0.0,0.0,1.6,12,1,7,24,165,179400,12,1,20,42,190,3,390,35,1,1,0.1,0.0,0.001200,0.016980,0.001000,0.030435,0.000000,0.000000,0.000000,6.9,0.5,6,2.9
1,29,8.2,6.8,6.7,3.1,0.0,1.6,20,0,28,7,242,59700,13,0,43,28,192,4,390,67,0,1,4.3,0.0,0.027998,0.074074,0.049689,0.693069,0.055319,0.023256,0.010811,12.0,5.7,5,5.2
2,23,13.3,9.7,9.6,0.0,0.8,1.6,18,1,27,7,136,139100,17,0,35,22,9,5,390,28,1,0,8.4,0.1,0.237154,0.195440,0.236220,1.250000,0.353191,1.818182,0.614286,13.3,7.1,1,3.6
3,13,10.6,8.5,8.4,0.0,1.0,1.6,12,0,11,21,272,118900,15,1,42,34,69,3,295,90,1,1,3.0,0.2,0.001200,0.030083,0.020192,0.360825,0.005556,0.000000,0.000000,12.4,4.9,3,3.9
4,17,11.8,5.9,5.8,0.0,0.9,0.8,16,0,21,35,10,59000,15,0,2,27,70,3,390,40,1,1,5.5,0.4,0.070217,0.268456,0.235756,0.498575,0.046809,0.078431,0.016216,15.7,2.7,2,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8923,25,9.2,8.1,8.0,2.3,0.0,1.6,18,1,38,12,96,75100,13,0,1,43,71,3,295,50,1,1,5.2,0.0,0.004561,0.033315,0.015364,0.250896,0.038298,0.000000,0.000000,10.9,7.5,4,2.8
8924,29,10.4,5.4,5.3,1.5,0.0,0.8,12,1,48,26,134,15200,14,1,12,36,72,3,390,65,1,1,2.1,0.0,0.004561,0.033315,0.015364,0.250896,0.038298,0.000000,0.000000,14.1,4.3,3,8.7
8925,19,7.7,4.2,4.1,1.6,0.0,1.6,15,0,32,40,270,194100,11,1,19,3,131,4,295,62,0,0,3.0,0.0,0.020281,0.027162,0.014428,0.384615,0.022222,0.023256,0.007143,13.2,3.0,6,9.0
8926,11,11.5,4.3,4.2,1.6,0.0,0.8,18,1,12,37,85,14400,12,1,2,26,135,5,295,43,1,0,3.0,0.0,0.011119,0.048622,0.019118,0.371156,0.066667,0.078431,0.014286,11.9,2.0,4,7.6


In [87]:
df_ok["根入れ深さ"] = df_ok["矢板の全長"] - df_ok["左右壁体高さ"]
df_error["根入れ深さ"] = df_error["矢板の全長"] - df_error["左右壁体高さ"]

In [89]:
df_error

,堤体延長,堤体幅,左右壁体高さ,矢板の全長,根入れ深さ,中詰土天端高さ,堤外区間地表面,堤内区間地表面,引張材間隔,引張材位置,在来地盤重量,在来地盤区分,在来地盤N値,在来地盤摩擦角,在来地盤粘着力,在来地盤変形係数,中詰め重量,中詰め区分,中詰めN値,中詰め摩擦角,中詰め粘着力,矢板型,矢板材料,引張材直径,引張材材料番号,腹起し材質,腹起し番号,堤外側水位_常時,堤内側水位_常時,せん断変形破壊,滑動、支持力,根入れ部の安定,遮水効果,矢板の耐力照査,タイロッドの耐力照査,腹起しの耐力照査
0,18,6.8,4.5,5.7,1.2,4.4,0.0,0.4,0.8,2.0,15,_Sandy,29,9,10,116100,17,_Sandy,42,13,249,4,_SY295,57,1,1,9,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,12.6,6.9,13.5,6.6,6.8,0.0,0.2,1.6,1.7,17,_Sandy,6,7,16,178600,18,_Clay,13,8,136,3,_SY295,90,0,1,3,5.2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13,11.8,7.1,9.0,1.9,7.0,2.1,0.0,0.8,3.1,14,_Sandy,49,23,4,15700,13,_Clay,2,18,21,5,_SY295,26,0,0,9,2.6,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,3.5,5.1,8.7,3.6,5.0,2.2,0.0,0.8,3.2,15,_Sandy,5,8,11,124800,17,_Sandy,20,40,264,5,_SY390,88,1,0,6,4.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21,8.8,6.6,7.4,0.8,6.5,1.9,0.0,1.6,4.3,14,_Sandy,47,25,0,178300,19,_Clay,50,41,87,3,_SY390,50,0,0,9,2.9,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,25,11.7,6.2,10.3,4.1,6.1,0.0,0.3,1.6,1.7,17,_Sandy,44,1,176,76500,15,_Sandy,49,33,247,4,_SY390,88,1,1,9,1.6,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068,13,10.0,5.4,8.1,2.7,5.3,1.8,0.0,1.6,3.0,14,_Clay,0,45,156,192100,17,_Sandy,26,16,66,5,_SY390,40,0,0,5,4.4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1069,23,9.2,5.0,7.6,2.6,4.9,1.7,0.0,1.6,2.6,19,_Sandy,27,6,0,166900,16,_Sandy,23,33,10,4,_SY295,64,1,0,1,2.8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1070,13,11.9,9.3,11.8,2.5,9.2,1.6,0.0,0.8,2.8,14,_Clay,2,43,260,95300,16,_Sandy,45,31,11,5,_SY390,53,1,1,2,2.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
#カテゴリ変数のカテゴリ化

def label_en(la,df):
    label = df[la]
    label = label.astype("category")
    le_embarked = LabelEncoder()
    le_embarked.fit(label)
    df[la] =  le_embarked.transform(label)
    return

#タイプ3:0，タイプ4:1，タイプ5:2
label_en('在来地盤区分',df_error)
label_en('中詰め区分',df_error)
label_en('矢板材料',df_error)
label_en('矢板型',df_error)
label_en('腹起し番号',df_error)
label_en('引張材材料番号',df_error)
label_en('腹起し材質',df_error)

label_en('在来地盤区分',df_ok)
label_en('中詰め区分',df_ok)
label_en('矢板材料',df_ok)
label_en('矢板型',df_ok)
label_en('腹起し番号',df_ok)
label_en('引張材材料番号',df_ok)
label_en('腹起し材質',df_ok)

In [98]:
df_error

,堤体延長,堤体幅,左右壁体高さ,矢板の全長,根入れ深さ,中詰土天端高さ,堤外区間地表面,堤内区間地表面,引張材間隔,引張材位置,在来地盤重量,在来地盤区分,在来地盤N値,在来地盤摩擦角,在来地盤粘着力,在来地盤変形係数,中詰め重量,中詰め区分,中詰めN値,中詰め摩擦角,中詰め粘着力,矢板型,矢板材料,引張材直径,引張材材料番号,腹起し材質,腹起し番号,堤外側水位_常時,堤内側水位_常時,せん断変形破壊,滑動、支持力,根入れ部の安定,遮水効果,矢板の耐力照査,タイロッドの耐力照査,腹起しの耐力照査
0,18,6.8,4.5,5.7,1.2,4.4,0.0,0.4,0.8,2.0,15,1,29,9,10,116100,17,1,42,13,249,1,0,57,1,1,8,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,12.6,6.9,13.5,6.6,6.8,0.0,0.2,1.6,1.7,17,1,6,7,16,178600,18,0,13,8,136,0,0,90,0,1,2,5.2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13,11.8,7.1,9.0,1.9,7.0,2.1,0.0,0.8,3.1,14,1,49,23,4,15700,13,0,2,18,21,2,0,26,0,0,8,2.6,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,3.5,5.1,8.7,3.6,5.0,2.2,0.0,0.8,3.2,15,1,5,8,11,124800,17,1,20,40,264,2,1,88,1,0,5,4.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21,8.8,6.6,7.4,0.8,6.5,1.9,0.0,1.6,4.3,14,1,47,25,0,178300,19,0,50,41,87,0,1,50,0,0,8,2.9,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,25,11.7,6.2,10.3,4.1,6.1,0.0,0.3,1.6,1.7,17,1,44,1,176,76500,15,1,49,33,247,1,1,88,1,1,8,1.6,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068,13,10.0,5.4,8.1,2.7,5.3,1.8,0.0,1.6,3.0,14,0,0,45,156,192100,17,1,26,16,66,2,1,40,0,0,4,4.4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1069,23,9.2,5.0,7.6,2.6,4.9,1.7,0.0,1.6,2.6,19,1,27,6,0,166900,16,1,23,33,10,1,0,64,1,0,0,2.8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1070,13,11.9,9.3,11.8,2.5,9.2,1.6,0.0,0.8,2.8,14,0,2,43,260,95300,16,1,45,31,11,2,1,53,1,1,1,2.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
df = pd.concat([df_ok,df_error],axis = 0)

## 安全率のラベリング

In [100]:
# th：しきい値
th = [0.8,1.0,1.2]

for s in safety_label:
    df.loc[df[s].isna(),f"{s}"+"_label"] = 3 #計算不可
    df.loc[df[s] < th[0], f"{s}"+"_label"] = 0
    df.loc[th[0] <= df[s], f"{s}"+"_label"] = 1 
    df.loc[th[1] < df[s], f"{s}"+"_label"] = 2
    df.loc[th[2] <= df[s] , f"{s}"+"_label"] = 3

In [107]:
pd.set_option('display.max_columns',40)
df

,堤体延長,堤体幅,左右壁体高さ,中詰土天端高さ,堤外区間地表面,堤内区間地表面,引張材間隔,在来地盤重量,在来地盤区分,在来地盤N値,在来地盤摩擦角,在来地盤粘着力,在来地盤変形係数,中詰め重量,中詰め区分,中詰めN値,中詰め摩擦角,中詰め粘着力,矢板型,矢板材料,...,堤外側水位_常時,堤内側水位_常時,せん断変形破壊,滑動、支持力,根入れ部の安定,遮水効果,矢板の耐力照査,タイロッドの耐力照査,腹起しの耐力照査,矢板の全長,引張材位置,腹起し番号,根入れ深さ,せん断変形破壊_label,滑動、支持力_label,根入れ部の安定_label,遮水効果_label,矢板の耐力照査_label,タイロッドの耐力照査_label,腹起しの耐力照査_label
0,20,5.7,4.0,3.9,0.0,0.0,1.6,12,1,7,24,165,179400,12,1,20,42,190,0,1,...,0.1,0.0,0.001200,0.016980,0.001000,0.030435,0.000000,0.000000,0.000000,6.9,0.5,5,2.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,29,8.2,6.8,6.7,3.1,0.0,1.6,20,0,28,7,242,59700,13,0,43,28,192,1,1,...,4.3,0.0,0.027998,0.074074,0.049689,0.693069,0.055319,0.023256,0.010811,12.0,5.7,4,5.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,23,13.3,9.7,9.6,0.0,0.8,1.6,18,1,27,7,136,139100,17,0,35,22,9,2,1,...,8.4,0.1,0.237154,0.195440,0.236220,1.250000,0.353191,1.818182,0.614286,13.3,7.1,0,3.6,0.0,0.0,0.0,3.0,0.0,3.0,0.0
3,13,10.6,8.5,8.4,0.0,1.0,1.6,12,0,11,21,272,118900,15,1,42,34,69,0,0,...,3.0,0.2,0.001200,0.030083,0.020192,0.360825,0.005556,0.000000,0.000000,12.4,4.9,2,3.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17,11.8,5.9,5.8,0.0,0.9,0.8,16,0,21,35,10,59000,15,0,2,27,70,0,1,...,5.5,0.4,0.070217,0.268456,0.235756,0.498575,0.046809,0.078431,0.016216,15.7,2.7,1,9.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,25,11.7,6.2,6.1,0.0,0.3,1.6,17,1,44,1,176,76500,15,1,49,33,247,1,1,...,1.6,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.3,1.7,8,4.1,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1068,13,10.0,5.4,5.3,1.8,0.0,1.6,14,0,0,45,156,192100,17,1,26,16,66,2,1,...,4.4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.1,3.0,4,2.7,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1069,23,9.2,5.0,4.9,1.7,0.0,1.6,19,1,27,6,0,166900,16,1,23,33,10,1,0,...,2.8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.6,2.6,0,2.6,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1070,13,11.9,9.3,9.2,1.6,0.0,0.8,14,0,2,43,260,95300,16,1,45,31,11,2,1,...,2.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.8,2.8,1,2.5,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [106]:
df.to_csv(f"{data_folder}" + "input20001_30000_input+error.csv",encoding="shift-jis",index=None)